In [1]:
import os
os.getcwd()

'C:\\Users\\User\\Documents\\Studia\\NLP\\project2\\src'

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use("fivethirtyeight")

In [3]:
name = 'own_cnn_bow'
data = '../data/own_mod.parquet'

## 1: Load data

In [4]:
df = pd.read_parquet(data)
df

text  score
0     description say maybe cream cheese cream chees...      1
1     terrible followed recipe exactly tasted fine a...      1
2     im always amazed people rate recipe state chan...      1
3     didnt like creamy rich didnt feel good sour cr...      1
4              even using green chilies much hot u like      1
...                                                 ...    ...
1407  good omitted green chilies dont like lot heat ...      5
1408  cut calorie fat way using nonfat sour cream su...      5
1409  wonderful recipe used oz light cream cheese in...      5
1410  good thing would leave salt use lower sodium b...      5
1411  quick easy make family asks time dont add gree...      5

[1412 rows x 2 columns]

### 2.3 Feature Extraction

#### 2.3.1 Bag of words

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features= 32*32)

X = vectorizer.fit_transform(df.text)
X

<1412x1024 sparse matrix of type '<class 'numpy.int64'>'
	with 22836 stored elements in Compressed Sparse Row format>

## 2.4 Split Dataset

In [6]:
Y = np.array([[y>1, y>2, y>3, y>4] for y in df.score])

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

del X
del Y

In [8]:
print("X_train", X_train.shape)
print("y_train", y_train.shape)

X_train (1129, 1024)
y_train (1129, 4)


In [9]:
print("X_test", X_test.shape)
print("y_test", y_test.shape)

X_test (283, 1024)
y_test (283, 4)


## 3 Build model

In [10]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers

In [11]:
model_input = layers.Input(X_train.shape[1], sparse=True)
l = layers.Lambda(tf.sparse.to_dense)(model_input)
l = layers.Reshape((32, 32, 1))(l)
l = layers.Conv2D(32, (3, 3), activation='relu')(l)
l = layers.MaxPooling2D((2, 2))(l)
l = layers.Conv2D(64, (3, 3), activation='relu')(l)
l = layers.MaxPooling2D((2, 2))(l)
l = layers.Conv2D(64, (3, 3), activation='relu')(l)
l = layers.Flatten()(l)
l = layers.Dense(64, activation='relu')(l)
model_output = layers.Dense(4, activation='sigmoid')(l)

model = keras.Model(model_input, model_output)

In [12]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1024)]            0         
                                                                 
 lambda (Lambda)             (None, None)              0         
                                                                 
 reshape (Reshape)           (None, 32, 32, 1)         0         
                                                                 
 conv2d (Conv2D)             (None, 30, 30, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496     
                                                             

In [13]:
def score_accuracy(y_true, y_pred):
    return tf.math.reduce_sum(y_true, axis=1) == tf.math.round(tf.math.reduce_sum(y_pred, axis=1))

def score_distance(y_true, y_pred):
    return tf.math.abs(tf.math.reduce_sum(y_true, axis=1) - tf.math.round(tf.math.reduce_sum(y_pred, axis=1)))

In [14]:
model.compile(optimizer = tf.keras.optimizers.Adam(), loss = 'binary_crossentropy', metrics=[score_accuracy, score_distance])

In [15]:
def convert_sparse_matrix_to_sparse_tensor(X):
    coo = X.tocoo()
    indices = np.mat([coo.row, coo.col]).transpose()
    return tf.SparseTensor(indices, coo.data, coo.shape)

X_train = convert_sparse_matrix_to_sparse_tensor(X_train)
X_test = convert_sparse_matrix_to_sparse_tensor(X_test)

In [16]:
X_train = tf.sparse.reorder(X_train)
X_test = tf.sparse.reorder(X_test)

In [17]:
hist = model.fit(
    X_train,
    y_train,
    batch_size=32,
    epochs=50,
    validation_data = (X_test, y_test)
)

Epoch 1/50
36/36 [==============================] - 9s 20ms/step - loss: 0.6001 - score_accuracy: 0.2108 - score_distance: 1.2019 - val_loss: 0.5738 - val_score_accuracy: 0.2191 - val_score_distance: 1.1837
Epoch 2/50
36/36 [==============================] - 0s 8ms/step - loss: 0.5527 - score_accuracy: 0.2135 - score_distance: 1.1709 - val_loss: 0.5371 - val_score_accuracy: 0.2509 - val_score_distance: 1.0919
Epoch 3/50
36/36 [==============================] - 0s 8ms/step - loss: 0.4963 - score_accuracy: 0.2569 - score_distance: 1.0115 - val_loss: 0.5251 - val_score_accuracy: 0.3004 - val_score_distance: 1.0283
Epoch 4/50
36/36 [==============================] - 0s 8ms/step - loss: 0.4708 - score_accuracy: 0.2755 - score_distance: 0.9557 - val_loss: 0.4994 - val_score_accuracy: 0.2756 - val_score_distance: 1.0106
Epoch 5/50
36/36 [==============================] - 0s 9ms/step - loss: 0.4260 - score_accuracy: 0.3313 - score_distance: 0.8592 - val_loss: 0.5348 - val_score_accuracy: 0.247

In [23]:
model.save(f'../models/{name}')

INFO:tensorflow:Assets written to: ../models/own_cnn_bow\assets


INFO:tensorflow:Assets written to: ../models/own_cnn_bow\assets


In [24]:
hist_df = pd.DataFrame(hist.history)
hist_df

loss  score_accuracy  score_distance  val_loss  val_score_accuracy  \
0   0.600137        0.210806        1.201949  0.573791            0.219081   
1   0.552702        0.213463        1.170948  0.537076            0.250883   
2   0.496291        0.256864        1.011515  0.525097            0.300353   
3   0.470756        0.275465        0.955713  0.499423            0.275618   
4   0.425990        0.331267        0.859167  0.534796            0.247350   
5   0.392849        0.357839        0.795394  0.497995            0.346290   
6   0.367099        0.377325        0.751107  0.479204            0.328622   
7   0.333939        0.426926        0.682905  0.483622            0.346290   
8   0.304303        0.489814        0.596103  0.484836            0.339223   
9   0.280020        0.519043        0.558016  0.490467            0.371025   
10  0.255403        0.567759        0.490700  0.536476            0.371025   
11  0.223705        0.616475        0.436670  0.547007            0.381625   
12  0.204832        0.655447        0.392383  0.591869            0.371025   
13  0.175527        0.696191        0.336581  0.609259            0.381625   
14  0.150406        0.746678        0.277236  0.641905            0.413428   
15  0.119624        0.805137        0.215235  0.742279            0.374558   
16  0.104459        0.831710        0.185120  0.759300            0.424028   
17  0.088566        0.862710        0.147033  0.857383            0.416961   
18  0.081693        0.875111        0.135518  0.883323            0.420495   
19  0.067364        0.901683        0.107174  0.960230            0.431095   
20  0.053210        0.914969        0.091231  0.977450            0.402827   
21  0.040513        0.948627        0.057573  1.121645            0.409894   
22  0.036429        0.962799        0.042516  1.120130            0.427562   
23  0.027513        0.972542        0.033658  1.208490            0.402827   
24  0.025719        0.968999        0.037201  1.248616            0.427562   
25  0.020753        0.983171        0.022143  1.357231            0.448763   
26  0.019565        0.981399        0.024801  1.356151            0.424028   
27  0.015964        0.984057        0.019486  1.452053            0.434629   
28  0.014295        0.988485        0.018601  1.467169            0.427562   
29  0.011435        0.986714        0.015943  1.540443            0.388693   
30  0.010891        0.986714        0.017715  1.588038            0.427562   
31  0.013436        0.988485        0.015058  1.593560            0.424028   
32  0.030819        0.953942        0.051373  1.505199            0.431095   
33  0.044668        0.938884        0.067316  1.459011            0.438163   
34  0.042476        0.934455        0.069973  1.379976            0.434629   
35  0.051504        0.927369        0.080602  1.380398            0.413428   
36  0.024998        0.968999        0.034544  1.462166            0.431095   
37  0.014188        0.984942        0.019486  1.545532            0.416961   
38  0.012472        0.989371        0.015058  1.651297            0.431095   
39  0.010901        0.990257        0.013286  1.663057            0.445230   
40  0.009792        0.990257        0.012400  1.722210            0.445230   
41  0.009456        0.991143        0.012400  1.738912            0.438163   
42  0.008211        0.992028        0.010629  1.766007            0.445230   
43  0.008218        0.991143        0.011515  1.810906            0.445230   
44  0.008434        0.992028        0.011515  1.837737            0.420495   
45  0.008859        0.992028        0.011515  1.861000            0.420495   
46  0.008448        0.992028        0.011515  1.862148            0.413428   
47  0.009784        0.990257        0.012400  1.887074            0.427562   
48  0.008394        0.991143        0.011515  1.905251            0.455830   
49  0.007611        0.990257        0.012400  1.957260            0.427562   

    val_score_distance  
0   

In [20]:
hist_df.to_csv(f'../logs/{name}.csv')

In [22]:
plt.title('CNN on BoW data')

plt.plot(hist_df.score_accuracy, label='Training')
plt.plot(hist_df.val_score_accuracy, label='Validation')

plt.legend()
plt.xlabel('epoch')
plt.ylabel('accuracy')

plt.savefig(f'../figs/{name}.png')
plt.show()